In [ ]:
%load_ext Cython

manggling/demanggling
*pointer: access value

In [ ]:
%%cython --annotate --force
  
# cython: language_level=3
# cython: cdivision=True
# cython: boundscheck=False
# cython: wraparound=False
# cython: profile=False
# cython: linetrace=False
# cython: binding=False
# distutils: define_macros=CYTHON_TRACE_NOGIL=0

cimport numpy as np
import numpy as np
from libc.math cimport fabs
from cython.parallel cimport prange
from cython cimport floating, integral
cimport cython

cdef floating _l1_distance(
    floating[::1] X_a_row,
    floating[::1] X_b_row,
    integral n_features,
) nogil:
    cdef:
        int i
        floating dist = 0
    
    for i in range(n_features):
        dist += fabs(X_a_row[i] - X_b_row[i])
        
    return dist

cdef void _pairwise_dist(
    floating[:, ::1] X_a, # IN
    floating[:, ::1] X_b, # IN
    floating[:, ::1] distances, # OUT
) nogil:
    cdef:
        int i, j
        int n_rows_X_a = X_a.shape[0]
        int n_rows_X_b = X_b.shape[0]
        int n_features = X_a.shape[1]
        
    for i in prange(n_rows_X_a, nogil=True):
        for j in range(n_rows_X_b):
            distances[i, j] = _l1_distance(X_a[i], X_b[j], n_features)

def pairwise_dist(
    floating[:, ::1] X_a,
    floating[:, ::1] X_b
):
    float_dtype = np.float32 if floating is float else np.float64
    cdef:
        floating[:, ::1] distances = np.zeros([X_a.shape[0], X_b.shape[0]], dtype=float_dtype)
    
    _pairwise_dist(X_a, X_b, distances)
    
    return np.asarray(distances)

In [ ]:
def time_func(func, *args, **kwargs):
    times = []
    for _ in range(10):
        start = time.perf_counter()
        expected = func(*args, **kwargs)
        end = time.perf_counter()
        time_elapsed = end - start
        times.append(time_elapsed)
    mean_time = np.mean(times)
    return expected, mean_time

In [ ]:
from scipy.spatial.distance import cdist
import time 
def test_correctness(n, p, metric="cityblock"):
    X_a = np.random.rand(n, p)
    X_b = np.random.rand(n, p)
    
    expected, t1 = time_func(cdist, X_a, X_b, metric=metric)
    actual, t2 = time_func(pairwise_dist, X_a, X_b)
    print(f"t1={t1}s")
    print(f"t2={t2}s")

    np.testing.assert_allclose(actual, expected, verbose=True)

In [ ]:
test_correctness(5000, 100)